# Transfer Learning and PreTrained Models

Before we get into building a ton of models, I want to first talk about one of the coolest aspects of Deep Learning: **Transfer Learning**. As we have seen in the [Intro to PyTorch](https://github.com/priyammaz/HAL-DL-From-Scratch/tree/main/Intro%20to%20PyTorch), a Deep Learning model is really just a structured bag of parameters. In the Linear Regression example, we had only two parameters, a weight $W_1$ and a Bias $W_0$. The only models we can create by multiplying and adding numbers together is a linear model. The Dense MNIST model we then made after was a deeper model with more parameters and connections, but we were still only multiplying and adding numbers together, therefore we can only represent a linear relationship. To fix this we then added in some ReLU (Rectified Linear Unit) activation functions that would allow us to introduce non-linearity to our model! All these weights that are randomly initialized when we define the model are then optimized and tuned so we have the least error between our predictions and true value. 

If we took our model and saved the parameters that define it, this would be known as a **pre-trained** model. Transfer learning is then the idea that, if we already have a model that can predict handwritten digits very well (0 through 9), and we have a new dataset of the lowercase alphabet (a through z), do we really need to train a new model from scratch? Or can we just grab our pre-trained model and its already optimized weights and finetune it to the new dataset?

Intuitively you can think about it this way. Lets say we show a kid some pictures of dogs and cats (like the dataset we are working on). After we show them some images of these dogs and cats, they will be able to classify them pretty well all on their own! Now we want to give them a new task, classify between tigers and wolves. Obviously tigers are not the same as cats and wolves are not the same as dogs, but they do share a bunch of similarities right? So the kid would then use their previous knowledge (The **pre-trained** model) and exapand it to be able to do this new task.

## AlexNet and Convolutions
We will be skipping ahead a bit here. In the next lesson, PyTorch for Vision, we will be covering in detail the ideas for convolutions and why they are much better that Dense Linear layers for Images. For now, pretend the model we will use is some black box that takes in an image, and outputs some probabilities of belonging to different classes. The specific model we will look at is AlexNet, which was probably the first real evidence of the power of deep learning in 2012. It was trained on the ImageNet task (predict across 1000 classes of images) and beat all other methods by a significant margin. We will be loading a PyTorch defined version of this model, but don't worry, we will implement this entire model from scratch in the next lesson! I am just trying to offer some intuition for Deep Learning before we get into the weeds!


## More Intution about Deep Learning 

As we have seen, the reason it is called "Deep" Learning is because the model physically has depth and many layers of computation. Because of this we actually get some interesting properties!

![Image](https://anhvnn.files.wordpress.com/2018/02/1i75ghqla1cbkeyqz3j-yga.jpeg)

[credit](https://anhvnn.wordpress.com/2018/02/01/deep-learning-computer-vision-and-convolutional-neural-networks/)

This is a common image you will see when exploring Deep Learning. You can see that earlier features that are learned are focused on simple things to find in an image, like edges, lines, etc... Regardless of the Image problem at hand, we always have to detect these features, so the underlying weights in a pretrained model that encode this can easily be used. As we move forward through the model, the extracted features in the image become much more specific to the dataset, and in this case there is some type of face detection. It is really cool though that convolutions (Again a black box that does some type of image processing) can extract abstract features at such a high level!

Now lets say we want to use this pretrained model and now detect Dogs instead. Well on its own, this model would do poorly as it is optimized to find human faces! But also the intial layers that do low level image feautures are probably fine to leave alone. Therefore, the typical strategy is to keep the earlier parts of the model static (don't allow gradient updates) and then finetune the later layers.

There are some benefits for this:
- We can pretrain a model on a giant dataset, and then finetune it to a similar problem that we have limited data for
- We can control and avoid problems like overfitting better

In [11]:
### Imports ###
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import AlexNet, AlexNet_Weights
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

### Recap From Before

We will be skipping the details of the setup for the DataLoader and everything here. If you have any confusion, go back to my [PyTorch DataLoader](https://github.com/priyammaz/HAL-DL-From-Scratch/tree/main/PyTorch%20DataLoaders) tutorial where I go in depth about how this works!

In [18]:
### Build Cats vs Dogs Dataset ###
PATH_TO_DATA = "../data/PetImages/"

### DEFINE TRANSFORMATIONS ###
normalizer = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]) ### IMAGENET MEAN/STD ###
train_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalizer
                                      ])


dataset = ImageFolder(PATH_TO_DATA, transform=train_transforms)

train_samples, test_samples = int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, lengths=[train_samples, test_samples])

### Lets Now Load the Model!!
We will now load the AlexNet model from PyTorch and then poke around a bit to see how these models are typically defined! Below you will see a ton of thigns you haven't seen before but its ok! We will go into detail again in the next tutorial. For now lets just take a rought look at whats going on:

In [20]:
model = AlexNet()
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

All PyTorch models are saved like this, where every layer has a name and we can access them individually. For example, if we want to look at the 1st linear layer in the classifier we can do it this way:

```
model.classifier[1]
```

In [21]:
model.classifier[1]

Linear(in_features=9216, out_features=4096, bias=True)

### What is the Output of the Model?

Notice the very last linear layer looks like this:

```
Linear(in_features=4096, out_features=1000, bias=True)
```

We can clearly see here that the model is outputing 1000 values because the ImageNet task requires us to predict across 1000 classes. As we start thinking about how we want to use this model, if we have a binary classification problem (Cats vs Dogs), then we will need to change this last linear layer so its output only 2 values.

**Lets Update the Model!**
We know that we can access the last layer of the model by doing *model.classifier[6]* and we want to change this to a linear layer. We saw in the [Intro to PyTorch](https://github.com/priyammaz/HAL-DL-From-Scratch/tree/main/Intro%20to%20PyTorch) that we can define a linear layer as:

```
Linear(in_features, out_features)
```

but we have to make sure that the input features of the new layer matches the output features of the previous Linear layer. We can see in the 4th Linear layer that the output is 4096 so we will ensure to keep that as our input!

In [23]:
model = AlexNet()
model.classifier[6] = nn.Linear(4096, 2)

model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Now we can clearly see that our model is outputting 2 features rather than 1000! This then gives us everything we need, so lets pass a test tesor through it to make sure its all functional, and then we can do a bit more exploration. As we said before, the common shape for Image data is:

```
[Batch Size x Channels x Image Height x Image Width]
```

We will just make a dummy tensor that matches the CatsVsDogs dataset we made that will have the shape [16 x 3 224 x 224]

In [27]:
rand_data = torch.rand(16,3,224,224)
model_output = model(rand_data)
model_output.shape

torch.Size([16, 2])

We can see that we passed in 16 images and the output is 2 classes per image, so the model is fully functional!!

### Checking out the Model Parameters 
The attribute **.named_parameters()** that a model has allows us to iterate through all the names and parameters of the model! You will notice that the parameters are also stored as Tensors and these are the number that are updated through optimization! We can add up the number of parameters in the model and see that after we reduced the final output dimension to 2, we have roughly 57 Million parameters! That may sound like a lot but most Deep Learning models today have Billions of parameters.

In [47]:
total_parameters = 0
for name, params in model.named_parameters():
    num_params = int(torch.prod(torch.tensor(params.shape)))
    print(name,":", params.shape, "Num Parameters:", num_params)
    total_parameters += num_params
    
print("------------------------")
print("Total Parameters in Model", total_parameters)
    

features.0.weight : torch.Size([64, 3, 11, 11]) Num Parameters: 23232
features.0.bias : torch.Size([64]) Num Parameters: 64
features.3.weight : torch.Size([192, 64, 5, 5]) Num Parameters: 307200
features.3.bias : torch.Size([192]) Num Parameters: 192
features.6.weight : torch.Size([384, 192, 3, 3]) Num Parameters: 663552
features.6.bias : torch.Size([384]) Num Parameters: 384
features.8.weight : torch.Size([256, 384, 3, 3]) Num Parameters: 884736
features.8.bias : torch.Size([256]) Num Parameters: 256
features.10.weight : torch.Size([256, 256, 3, 3]) Num Parameters: 589824
features.10.bias : torch.Size([256]) Num Parameters: 256
classifier.1.weight : torch.Size([4096, 9216]) Num Parameters: 37748736
classifier.1.bias : torch.Size([4096]) Num Parameters: 4096
classifier.4.weight : torch.Size([4096, 4096]) Num Parameters: 16777216
classifier.4.bias : torch.Size([4096]) Num Parameters: 4096
classifier.6.weight : torch.Size([2, 4096]) Num Parameters: 8192
classifier.6.bias : torch.Size([2]

### Lets Train This Model!